## Дополнительное задание (курс "Практикум по программированию на языке Python")

### Тема: Web-сервер для Telegram-бота.

#### Преподаватели: Мурат Апишев (mel-lain@yandex.ru) и Роман Ищенко (roman.ischenko@gmail.com)

**Выдана**:   -

**Дедлайн**:   -

**Среда выполнения**: Jupyter Notebook (Python 3.7)

#### Правила:

Результаты выполнения задания:

- архив со скриптами и файлами Dockerfile, Pipfile и Pipfile.lock, который 1-2 команды позволяет развернуть сервер, решающий поставленные в задании задачи
- Jupyter Notebook, где __весь код__ из скриптов дублируется (1 ячейка - 1 скрипт) с комментарием, содержащим информацию о том, из какого файла взят код и что верхнеуровнево этот код делает
- ссылка на чат-бота в Telegram, который автоматически начинает обработку пользовательских запросов после разворачивания сервера

__Максимальное число баллов за задание - 25__.

Работа делится на базовую и дополнительную части, __полное выполнение базовой является обязательным__! Без этого вся работа автоматически оцениваниется 0 баллов. Дополнительную часть можно решать фрагментарно, оценка будет выставляться пропорционально.

Готовое задание отправляется на почту преподавателя.

Задание выполняется самостоятельно. Если какие-то студенты будут уличены в списывании, все они автоматически получат за эту работу 0 баллов. Если вы нашли в Интернете какой-то специфичный код, который собираетесь заимствовать, обязательно укажите это в задании - наверняка вы не единственный, кто найдёт и использует эту информацию.

Удалять фрагменты формулировок заданий запрещается.

#### Постановка задачи:

- В данной работе нужно написать веб-сервер для обслуживания чат-бота в мессенджере Telegram
- Чат-бот предназначен для обработки изображений, передаваемых пользователем
- Сервер должен быть реализован на Flask + waitress
- Проект разворачивается с помощью библиотеки управления виртуальными окружениями Pipenv и технологии контейнеризации Docker

### Базовая часть (12 баллов)

- Требуется реализовать чат-бота для мессенджера Telegram, который возвращает пользователю на любое входное текстовое сообщение это сообщение. Предполагается, что никакие другие данные (картинки, стикеры) бот получать не будет
- Бэкенд проекта должен быть реализован на Flask + waitress (или FastAPI)
- Для прокидывания сервера наружу требуется использовать сервис и утилиту ngrok (как исполняемы файл, не pyngrok)
- Проект должен разворачиваться с помощью Docker, внутри которого создается виртуальное окружение с помощью Pipenv, решение должно разворачиваться командами:

```
docker build -t tg-bot .
docker run -d tg-bot
```

- __Обязательно (!!!)__ нужно вставить копии содержимого всех файлов проекта в этот ноутбук (1 файл - 1 ячейка) и подписать, к какому файлу относится какая ячейка и что код в ней в общих чертах делает

__Рекомендации:__

1. [Ссылка](https://core.telegram.org/bots) на инструкцию по созданию ботов в Telegram

2. Для работы с Telegram лучше всего использовать библиотеку python-telegram-bot

3. Рекомендуемая сигнатура для обработчика POST-запросов от бота:

In [ ]:
# app = Flask(__name__)

@app.route(f'/{BOT_TOKEN}', methods=['POST'])

4. Рекомендуемая структура проекта (плоская):
    - Dockerfile
    - Pipfile
    - Pipfile.lock
    - settings.py (для хранения токена, локального адреса сервера \[localhost + port\] и любых иных общих статических данных)
    - server.py (для старта локального сервера получения/отправки запросов)
    - configure_and_run.py (для настройки и старта всего сервиса)

    Опционально можно завести отдельные файлы для логики обработчика запросов, хранения текстов сообщений и любых иных целей


5. Dockerfile может иметь следующую структуру:

```
FROM python:3.7.9-stretch
# установка нужных утилит в систему, например, curl и unzip
# установка pipenv
WORKDIR /app
# копирование всех важных файлов в рабочую директорию
# запуск pipenv
ENTRYPOINT ["pipenv", "run", "python", "configure_and_run.py", "&"]
```

Последняя команда будет запускаться при вызове `docker run`

6. Вывод информации для отладки и логгирования в stdout можно делать с помощью `os.system(f'echo {message}')`

7. Поскольку локально поднятый сервер не виден снаружи, нужно как-то прокинуть его в Интернет, это можно сделать с помощью облачного хостинга или сервиса ngrok, который можно использовать через одноименную консольную утилиту, что и требуется в задании

8. Выкачивание и запуск ngrok можно выполнить в configure_and_run.py, в этом помогут утилиты curl и unzip. Для этих и других задач, требующих параллельного вызова сторонних приложений и скриптов, полезен модуль subprocess

9. После того, как ngrok будет запущен на указанном локальном адресе, нужно будет передать внешний адрес боту (выставить webhook). Достать этот адрес можно, запустив следующую команду и распарсив выходной файл (там будет json):


In [ ]:
os.system(f'curl http://localhost:4040/api/tunnels > {filename}')

10. После этого выставить webhook можно в server.py после создания и перед запуском Flask-сервера командой

In [ ]:
# bot = telegram.Bot(BOT_TOKEN)

bot.setWebhook('{URL}/{HOOK}'.format(URL=EXTERNAL_URL, HOOK=BOT_TOKEN))

11. Обработку запросов стоит оборачивать в try-except-блок и отправлять пользователю валидное сообщение об ошибке или просьбу повторить какое-то действие, саму ошибку при этому надо логгировать

12. Все вспомогательные файлы, возникающие в процессе работы, должны гарантированно удаляться, как только надобность в них отпадает

13. Запуск сервера имеет смысл производить из configure_and_run.py с помощью subprocess (полезен будет метод communicate)

In [38]:
%%writefile settings.py
BOT_TOKEN = "5375421732:AAElwJicyx-ut2RWykbVTrf0amqrUTtdQ88"
HOST = "0.0.0.0"
PORT = 8000
OUT_ADDR_FILENAME = "ngrok_addr.json"
TGZ_FILENAME = "ngrok.tgz"
NGROK_DOWNLOADING_URL = "https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz"
NGROK_AUTH_TOKEN = "29LKH3auC8TMYurjk3u6bgKRMI8_AAR7QsWFZ3DRRmCguEFB"

Overwriting settings.py


In [35]:
%%writefile server.py
import requests
import sys

from flask import Flask, request
from waitress import serve
import logging
import telegram
from telegram import Update
from settings import *

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

app = Flask(__name__)
bot = telegram.Bot(BOT_TOKEN)

@app.route(f"/{BOT_TOKEN}", methods=['POST'])
def echo() -> dict:
    # Создание объекта класса Update из данных, полученных с запроса
    update = Update.de_json(request.get_json(force=True), bot)

    # Достам chat_id и text, чтобы понять, кому и что отправлять
    chat_id = update.message.chat.id
    text = update.message.text
    
    try:
        bot.sendMessage(chat_id=chat_id, text=text)
    except Exception:
        bot.sendMessage(chat_id=chat_id, text="Wrong input")

    return {'ok': True}

if __name__ == '__main__':
    ngrok_url = sys.argv[1]
    
    # Даём боту понять, откуда прослушивать обновления
    bot.setWebhook('{URL}/{HOOK}'.format(URL=ngrok_url, HOOK=BOT_TOKEN))
    
    serve(app, host=HOST, port=PORT)

Overwriting server.py


In [54]:
%%writefile configure_and_run.py
import subprocess
import os
import requests
import json
from settings import *


# Установка ngrok
response = requests.get(NGROK_DOWNLOADING_URL)
with open(TGZ_FILENAME, "wb") as fd:
    fd.write(response.content)
    
os.system(f"tar xzf {TGZ_FILENAME}")
os.remove(TGZ_FILENAME)

subprocess.call(['./ngrok', 'config', 'add-authtoken', NGROK_AUTH_TOKEN])
first_proc = subprocess.Popen(args=["./ngrok", "http", str(PORT)], 
                              stdout=subprocess.DEVNULL)

# Ожидаем, пока дочерний процесс запустит сервер
try:
    first_proc.wait(timeout=1)
except subprocess.TimeoutExpired:
    pass

# Извлекаем публичный URL, который затем надо передать боту
os.mknod(OUT_ADDR_FILENAME)
os.system(f'curl http://localhost:4040/api/tunnels > {OUT_ADDR_FILENAME}')

with open(OUT_ADDR_FILENAME, "r") as fd:
    data = json.load(fd)
    ngrok_addr = data["tunnels"][0]["public_url"]
    
os.remove(OUT_ADDR_FILENAME)

# Запускаем веб-сервер
second_proc = subprocess.Popen(args=['python3', 'server.py', ngrok_addr])

Overwriting configure_and_run.py


In [55]:
%%writefile Dockerfile
FROM python:3.7.9-stretch

RUN pip install pipenv
RUN apt-get update && apt-get install -y curl
RUN apt-get install -y tar
RUN pipenv install

WORKDIR /app
COPY settings.py .
COPY server.py .
COPY configure_and_run.py .
ENTRYPOINT ["pipenv", "run", "python", "configure_and_run.py", "&"]

Overwriting Dockerfile


In [29]:
%%writefile Pipfile
[[source]]
url = "https://pypi.python.org/simple"
verify_ssl = true
name = "pypi"

[packages]
requests = "*"
flask = "*"
waitress = "*"
python-telegram-bot = "*"

[dev-packages]

[requires]
python_version = "3.8"

Overwriting Pipfile


In [31]:
%%writefile Pipfile.lock
{
    "_meta": {
        "hash": {
            "sha256": "0626490ff859fb7a19ea1507284830b08e95314deb08f3159127b50872ad15c6"
        },
        "pipfile-spec": 6,
        "requires": {
            "python_version": "3.8"
        },
        "sources": [
            {
                "name": "pypi",
                "url": "https://pypi.python.org/simple",
                "verify_ssl": true
            }
        ]
    },
    "default": {
        "apscheduler": {
            "hashes": [
                "sha256:3bb5229eed6fbbdafc13ce962712ae66e175aa214c69bed35a06bffcf0c5e244",
                "sha256:e8b1ecdb4c7cb2818913f766d5898183c7cb8936680710a4d3a966e02262e526"
            ],
            "version": "==3.6.3"
        },
        "backports.zoneinfo": {
            "hashes": [
                "sha256:17746bd546106fa389c51dbea67c8b7c8f0d14b5526a579ca6ccf5ed72c526cf",
                "sha256:1b13e654a55cd45672cb54ed12148cd33628f672548f373963b0bff67b217328",
                "sha256:1c5742112073a563c81f786e77514969acb58649bcdf6cdf0b4ed31a348d4546",
                "sha256:4a0f800587060bf8880f954dbef70de6c11bbe59c673c3d818921f042f9954a6",
                "sha256:5c144945a7752ca544b4b78c8c41544cdfaf9786f25fe5ffb10e838e19a27570",
                "sha256:7b0a64cda4145548fed9efc10322770f929b944ce5cee6c0dfe0c87bf4c0c8c9",
                "sha256:8439c030a11780786a2002261569bdf362264f605dfa4d65090b64b05c9f79a7",
                "sha256:8961c0f32cd0336fb8e8ead11a1f8cd99ec07145ec2931122faaac1c8f7fd987",
                "sha256:89a48c0d158a3cc3f654da4c2de1ceba85263fafb861b98b59040a5086259722",
                "sha256:a76b38c52400b762e48131494ba26be363491ac4f9a04c1b7e92483d169f6582",
                "sha256:da6013fd84a690242c310d77ddb8441a559e9cb3d3d59ebac9aca1a57b2e18bc",
                "sha256:e55b384612d93be96506932a786bbcde5a2db7a9e6a4bb4bffe8b733f5b9036b",
                "sha256:e81b76cace8eda1fca50e345242ba977f9be6ae3945af8d46326d776b4cf78d1",
                "sha256:e8236383a20872c0cdf5a62b554b27538db7fa1bbec52429d8d106effbaeca08",
                "sha256:f04e857b59d9d1ccc39ce2da1021d196e47234873820cbeaad210724b1ee28ac",
                "sha256:fadbfe37f74051d024037f223b8e001611eac868b5c5b06144ef4d8b799862f2"
            ],
            "markers": "python_version >= '3.6' and python_version < '3.9'",
            "version": "==0.2.1"
        },
        "cachetools": {
            "hashes": [
                "sha256:2cc0b89715337ab6dbba85b5b50effe2b0c74e035d83ee8ed637cf52f12ae001",
                "sha256:61b5ed1e22a0924aed1d23b478f37e8d52549ff8a961de2909c69bf950020cff"
            ],
            "version": "==4.2.2"
        },
        "certifi": {
            "hashes": [
                "sha256:6ae10321df3e464305a46e997da41ea56c1d311fb9ff1dd4e04d6f14653ec63a",
                "sha256:8d15a5a7fde18536a249c49e07e8e462b8fc13de21b3c80e8a68315dfa227c99"
            ],
            "version": "==2022.5.18"
        },
        "charset-normalizer": {
            "hashes": [
                "sha256:2857e29ff0d34db842cd7ca3230549d1a697f96ee6d3fb071cfa6c7393832597",
                "sha256:6881edbebdb17b39b4eaaa821b438bf6eddffb4468cf344f09f89def34a8b1df"
            ],
            "markers": "python_version >= '3'",
            "version": "==2.0.12"
        },
        "click": {
            "hashes": [
                "sha256:7682dc8afb30297001674575ea00d1814d808d6a36af415a82bd481d37ba7b8e",
                "sha256:bb4d8133cb15a609f44e8213d9b391b0809795062913b383c62be0ee95b1db48"
            ],
            "version": "==8.1.3"
        },
        "flask": {
            "hashes": [
                "sha256:315ded2ddf8a6281567edb27393010fe3406188bafbfe65a3339d5787d89e477",
                "sha256:fad5b446feb0d6db6aec0c3184d16a8c1f6c3e464b511649c8918a9be100b4fe"
            ],
            "index": "pypi",
            "version": "==2.1.2"
        },
        "idna": {
            "hashes": [
                "sha256:84d9dd047ffa80596e0f246e2eab0b391788b0503584e8945f2368256d2735ff",
                "sha256:9d643ff0a55b762d5cdb124b8eaa99c66322e2157b69160bc32796e824360e6d"
            ],
            "markers": "python_version >= '3'",
            "version": "==3.3"
        },
        "importlib-metadata": {
            "hashes": [
                "sha256:1208431ca90a8cca1a6b8af391bb53c1a2db74e5d1cef6ddced95d4b2062edc6",
                "sha256:ea4c597ebf37142f827b8f39299579e31685c31d3a438b59f469406afd0f2539"
            ],
            "markers": "python_version < '3.10'",
            "version": "==4.11.3"
        },
        "itsdangerous": {
            "hashes": [
                "sha256:2c2349112351b88699d8d4b6b075022c0808887cb7ad10069318a8b0bc88db44",
                "sha256:5dbbc68b317e5e42f327f9021763545dc3fc3bfe22e6deb96aaf1fc38874156a"
            ],
            "version": "==2.1.2"
        },
        "jinja2": {
            "hashes": [
                "sha256:31351a702a408a9e7595a8fc6150fc3f43bb6bf7e319770cbc0db9df9437e852",
                "sha256:6088930bfe239f0e6710546ab9c19c9ef35e29792895fed6e6e31a023a182a61"
            ],
            "version": "==3.1.2"
        },
        "markupsafe": {
            "hashes": [
                "sha256:0212a68688482dc52b2d45013df70d169f542b7394fc744c02a57374a4207003",
                "sha256:089cf3dbf0cd6c100f02945abeb18484bd1ee57a079aefd52cffd17fba910b88",
                "sha256:10c1bfff05d95783da83491be968e8fe789263689c02724e0c691933c52994f5",
                "sha256:33b74d289bd2f5e527beadcaa3f401e0df0a89927c1559c8566c066fa4248ab7",
                "sha256:3799351e2336dc91ea70b034983ee71cf2f9533cdff7c14c90ea126bfd95d65a",
                "sha256:3ce11ee3f23f79dbd06fb3d63e2f6af7b12db1d46932fe7bd8afa259a5996603",
                "sha256:421be9fbf0ffe9ffd7a378aafebbf6f4602d564d34be190fc19a193232fd12b1",
                "sha256:43093fb83d8343aac0b1baa75516da6092f58f41200907ef92448ecab8825135",
                "sha256:46d00d6cfecdde84d40e572d63735ef81423ad31184100411e6e3388d405e247",
                "sha256:4a33dea2b688b3190ee12bd7cfa29d39c9ed176bda40bfa11099a3ce5d3a7ac6",
                "sha256:4b9fe39a2ccc108a4accc2676e77da025ce383c108593d65cc909add5c3bd601",
                "sha256:56442863ed2b06d19c37f94d999035e15ee982988920e12a5b4ba29b62ad1f77",
                "sha256:671cd1187ed5e62818414afe79ed29da836dde67166a9fac6d435873c44fdd02",
                "sha256:694deca8d702d5db21ec83983ce0bb4b26a578e71fbdbd4fdcd387daa90e4d5e",
                "sha256:6a074d34ee7a5ce3effbc526b7083ec9731bb3cbf921bbe1d3005d4d2bdb3a63",
                "sha256:6d0072fea50feec76a4c418096652f2c3238eaa014b2f94aeb1d56a66b41403f",
                "sha256:6fbf47b5d3728c6aea2abb0589b5d30459e369baa772e0f37a0320185e87c980",
                "sha256:7f91197cc9e48f989d12e4e6fbc46495c446636dfc81b9ccf50bb0ec74b91d4b",
                "sha256:86b1f75c4e7c2ac2ccdaec2b9022845dbb81880ca318bb7a0a01fbf7813e3812",
                "sha256:8dc1c72a69aa7e082593c4a203dcf94ddb74bb5c8a731e4e1eb68d031e8498ff",
                "sha256:8e3dcf21f367459434c18e71b2a9532d96547aef8a871872a5bd69a715c15f96",
                "sha256:8e576a51ad59e4bfaac456023a78f6b5e6e7651dcd383bcc3e18d06f9b55d6d1",
                "sha256:96e37a3dc86e80bf81758c152fe66dbf60ed5eca3d26305edf01892257049925",
                "sha256:97a68e6ada378df82bc9f16b800ab77cbf4b2fada0081794318520138c088e4a",
                "sha256:99a2a507ed3ac881b975a2976d59f38c19386d128e7a9a18b7df6fff1fd4c1d6",
                "sha256:a49907dd8420c5685cfa064a1335b6754b74541bbb3706c259c02ed65b644b3e",
                "sha256:b09bf97215625a311f669476f44b8b318b075847b49316d3e28c08e41a7a573f",
                "sha256:b7bd98b796e2b6553da7225aeb61f447f80a1ca64f41d83612e6139ca5213aa4",
                "sha256:b87db4360013327109564f0e591bd2a3b318547bcef31b468a92ee504d07ae4f",
                "sha256:bcb3ed405ed3222f9904899563d6fc492ff75cce56cba05e32eff40e6acbeaa3",
                "sha256:d4306c36ca495956b6d568d276ac11fdd9c30a36f1b6eb928070dc5360b22e1c",
                "sha256:d5ee4f386140395a2c818d149221149c54849dfcfcb9f1debfe07a8b8bd63f9a",
                "sha256:dda30ba7e87fbbb7eab1ec9f58678558fd9a6b8b853530e176eabd064da81417",
                "sha256:e04e26803c9c3851c931eac40c695602c6295b8d432cbe78609649ad9bd2da8a",
                "sha256:e1c0b87e09fa55a220f058d1d49d3fb8df88fbfab58558f1198e08c1e1de842a",
                "sha256:e72591e9ecd94d7feb70c1cbd7be7b3ebea3f548870aa91e2732960fa4d57a37",
                "sha256:e8c843bbcda3a2f1e3c2ab25913c80a3c5376cd00c6e8c4a86a89a28c8dc5452",
                "sha256:efc1913fd2ca4f334418481c7e595c00aad186563bbc1ec76067848c7ca0a933",
                "sha256:f121a1420d4e173a5d96e47e9a0c0dcff965afdf1626d28de1460815f7c4ee7a",
                "sha256:fc7b548b17d238737688817ab67deebb30e8073c95749d55538ed473130ec0c7"
            ],
            "version": "==2.1.1"
        },
        "python-telegram-bot": {
            "hashes": [
                "sha256:534f5bb0ff4ca34c9252e97e0b3bcdab81d97be0eb4821682a361cb426c00e55",
                "sha256:baeff704baa2ac3dc17a944c02da888228ad258e89be2e5bcbd13a8a5102d573"
            ],
            "index": "pypi",
            "version": "==13.11"
        },
        "pytz": {
            "hashes": [
                "sha256:1e760e2fe6a8163bc0b3d9a19c4f84342afa0a2affebfaa84b01b978a02ecaa7",
                "sha256:e68985985296d9a66a881eb3193b0906246245294a881e7c8afe623866ac6a5c"
            ],
            "version": "==2022.1"
        },
        "pytz-deprecation-shim": {
            "hashes": [
                "sha256:8314c9692a636c8eb3bda879b9f119e350e93223ae83e70e80c31675a0fdc1a6",
                "sha256:af097bae1b616dde5c5744441e2ddc69e74dfdcb0c263129610d85b87445a59d"
            ],
            "version": "==0.1.0.post0"
        },
        "requests": {
            "hashes": [
                "sha256:68d7c56fd5a8999887728ef304a6d12edc7be74f1cfa47714fc8b414525c9a61",
                "sha256:f22fa1e554c9ddfd16e6e41ac79759e17be9e492b3587efa038054674760e72d"
            ],
            "index": "pypi",
            "version": "==2.27.1"
        },
        "six": {
            "hashes": [
                "sha256:1e61c37477a1626458e36f7b1d82aa5c9b094fa4802892072e49de9c60c4c926",
                "sha256:8abb2f1d86890a2dfb989f9a77cfcfd3e47c2a354b01111771326f8aa26e0254"
            ],
            "version": "==1.16.0"
        },
        "tornado": {
            "hashes": [
                "sha256:0a00ff4561e2929a2c37ce706cb8233b7907e0cdc22eab98888aca5dd3775feb",
                "sha256:0d321a39c36e5f2c4ff12b4ed58d41390460f798422c4504e09eb5678e09998c",
                "sha256:1e8225a1070cd8eec59a996c43229fe8f95689cb16e552d130b9793cb570a288",
                "sha256:20241b3cb4f425e971cb0a8e4ffc9b0a861530ae3c52f2b0434e6c1b57e9fd95",
                "sha256:25ad220258349a12ae87ede08a7b04aca51237721f63b1808d39bdb4b2164558",
                "sha256:33892118b165401f291070100d6d09359ca74addda679b60390b09f8ef325ffe",
                "sha256:33c6e81d7bd55b468d2e793517c909b139960b6c790a60b7991b9b6b76fb9791",
                "sha256:3447475585bae2e77ecb832fc0300c3695516a47d46cefa0528181a34c5b9d3d",
                "sha256:34ca2dac9e4d7afb0bed4677512e36a52f09caa6fded70b4e3e1c89dbd92c326",
                "sha256:3e63498f680547ed24d2c71e6497f24bca791aca2fe116dbc2bd0ac7f191691b",
                "sha256:548430be2740e327b3fe0201abe471f314741efcb0067ec4f2d7dcfb4825f3e4",
                "sha256:6196a5c39286cc37c024cd78834fb9345e464525d8991c21e908cc046d1cc02c",
                "sha256:61b32d06ae8a036a6607805e6720ef00a3c98207038444ba7fd3d169cd998910",
                "sha256:6286efab1ed6e74b7028327365cf7346b1d777d63ab30e21a0f4d5b275fc17d5",
                "sha256:65d98939f1a2e74b58839f8c4dab3b6b3c1ce84972ae712be02845e65391ac7c",
                "sha256:66324e4e1beede9ac79e60f88de548da58b1f8ab4b2f1354d8375774f997e6c0",
                "sha256:6c77c9937962577a6a76917845d06af6ab9197702a42e1346d8ae2e76b5e3675",
                "sha256:70dec29e8ac485dbf57481baee40781c63e381bebea080991893cd297742b8fd",
                "sha256:7250a3fa399f08ec9cb3f7b1b987955d17e044f1ade821b32e5f435130250d7f",
                "sha256:748290bf9112b581c525e6e6d3820621ff020ed95af6f17fedef416b27ed564c",
                "sha256:7da13da6f985aab7f6f28debab00c67ff9cbacd588e8477034c0652ac141feea",
                "sha256:8f959b26f2634a091bb42241c3ed8d3cedb506e7c27b8dd5c7b9f745318ddbb6",
                "sha256:9de9e5188a782be6b1ce866e8a51bc76a0fbaa0e16613823fc38e4fc2556ad05",
                "sha256:a48900ecea1cbb71b8c71c620dee15b62f85f7c14189bdeee54966fbd9a0c5bd",
                "sha256:b87936fd2c317b6ee08a5741ea06b9d11a6074ef4cc42e031bc6403f82a32575",
                "sha256:c77da1263aa361938476f04c4b6c8916001b90b2c2fdd92d8d535e1af48fba5a",
                "sha256:cb5ec8eead331e3bb4ce8066cf06d2dfef1bfb1b2a73082dfe8a161301b76e37",
                "sha256:cc0ee35043162abbf717b7df924597ade8e5395e7b66d18270116f8745ceb795",
                "sha256:d14d30e7f46a0476efb0deb5b61343b1526f73ebb5ed84f23dc794bdb88f9d9f",
                "sha256:d371e811d6b156d82aa5f9a4e08b58debf97c302a35714f6f45e35139c332e32",
                "sha256:d3d20ea5782ba63ed13bc2b8c291a053c8d807a8fa927d941bd718468f7b950c",
                "sha256:d3f7594930c423fd9f5d1a76bee85a2c36fd8b4b16921cae7e965f22575e9c01",
                "sha256:dcef026f608f678c118779cd6591c8af6e9b4155c44e0d1bc0c87c036fb8c8c4",
                "sha256:e0791ac58d91ac58f694d8d2957884df8e4e2f6687cdf367ef7eb7497f79eaa2",
                "sha256:e385b637ac3acaae8022e7e47dfa7b83d3620e432e3ecb9a3f7f58f150e50921",
                "sha256:e519d64089b0876c7b467274468709dadf11e41d65f63bba207e04217f47c085",
                "sha256:e7229e60ac41a1202444497ddde70a48d33909e484f96eb0da9baf8dc68541df",
                "sha256:ed3ad863b1b40cd1d4bd21e7498329ccaece75db5a5bf58cd3c9f130843e7102",
                "sha256:f0ba29bafd8e7e22920567ce0d232c26d4d47c8b5cf4ed7b562b5db39fa199c5",
                "sha256:fa2ba70284fa42c2a5ecb35e322e68823288a4251f9ba9cc77be04ae15eada68",
                "sha256:fba85b6cd9c39be262fcd23865652920832b61583de2a2ca907dbd8e8a8c81e5"
            ],
            "version": "==6.1"
        },
        "tzdata": {
            "hashes": [
                "sha256:238e70234214138ed7b4e8a0fab0e5e13872edab3be586ab8198c407620e2ab9",
                "sha256:8b536a8ec63dc0751342b3984193a3118f8fca2afe25752bb9b7fffd398552d3"
            ],
            "markers": "python_version >= '3.6'",
            "version": "==2022.1"
        },
        "tzlocal": {
            "hashes": [
                "sha256:89885494684c929d9191c57aa27502afc87a579be5cdd3225c77c463ea043745",
                "sha256:ee5842fa3a795f023514ac2d801c4a81d1743bbe642e3940143326b3a00addd7"
            ],
            "version": "==4.2"
        },
        "urllib3": {
            "hashes": [
                "sha256:44ece4d53fb1706f667c9bd1c648f5469a2ec925fcf3a776667042d645472c14",
                "sha256:aabaf16477806a5e1dd19aa41f8c2b7950dd3c746362d7e3223dbe6de6ac448e"
            ],
            "version": "==1.26.9"
        },
        "waitress": {
            "hashes": [
                "sha256:c549f5b2b4afd44d9d97d7cec79f3ef581e25d832827f415dc175327af674aa8",
                "sha256:e2e60576cf14a1539da79f7b7ee1e79a71e64f366a0b47db54a15e971f57bb16"
            ],
            "index": "pypi",
            "version": "==2.1.1"
        },
        "werkzeug": {
            "hashes": [
                "sha256:1ce08e8093ed67d638d63879fd1ba3735817f7a80de3674d293f5984f25fb6e6",
                "sha256:72a4b735692dd3135217911cbeaa1be5fa3f62bffb8745c5215420a03dc55255"
            ],
            "version": "==2.1.2"
        },
        "zipp": {
            "hashes": [
                "sha256:56bf8aadb83c24db6c4b577e13de374ccfb67da2078beba1d037c17980bf43ad",
                "sha256:c4f6e5bbf48e74f7a38e7cc5b0480ff42b0ae5178957d564d18932525d5cf099"
            ],
            "version": "==3.8.0"
        }
    },
    "develop": {}
}

Overwriting Pipfile.lock


### Дополнительная часть

__Задание 1 (2 балла):__ Доработать базовое решение до бота, который предлагает пользователю отправить изображение в формате JPG или PNG (в виде аттача), реагирует адекватно на попытки послать ему что-то иное, и в случае получения валидного изображения возвращает его в чат ответным сообщением

__Рекомендации:__

1. Нужно аккуратно обрабатывать крайние случаи (пришел файл, не являющийся изображением, или имеющий не тот формат)

2. Полезна функция, которая определяет, что сообщение содержит файл или фото:

In [ ]:
# content = telegram.Update...

@staticmethod
def is_file(content):
    return (content.message is not None and
            ((content.message.document is not None and content.message.document.file_id is not None) or
             (content.message.photo is not None and len(content.message.photo) and content.message.photo[0].file_id is not None)))

__Задание 2 (5 баллов):__ Модернизировать решение из предыдущего пункта - при получении изображения от пользователя нужно вернуть не его, а текстовое сообщение с кнопкой, в котором написано, что при нажатии кнопки полученное ранее изображение будет отправлено пользователю обратно. Это и должно происходить при нажатии на кнопку. При этом пользователь может отправить в ответ еще одно изображение, и тогда он снова получит сообщение с кнопкой, но сохранено будет уже новое изображение.

__Рекомендации:__

1. В отличие от предыдущих пунктов, здесь сессия пользователя начинает содержать состояния: пользователь еще не отправил картинку, и пользователь уже сделал это. Это означает, что стоит завести профиль пользователя, который будет идентифицироваться по `chat_id` (который используется при отправке сообщений) и содержать текущие состояние, картинку или любые иные полезные и вспомогательные данные пользователя

2. Хранить нужно не один общий профиль, а по профилю на ользователя, иначе сервис нельзя будет использоваться нескольким пользователям. Обычно профили хранятся в базах, для простоты реализации в задании достаточно ограничиться хранением в `dict` в оперативной памяти, который заводится при старте сервера

3. Нужно следить за корректностью изменения состояния сессии и профиля пользователя, в том числе и при возникновении различных ошибок

4. Полезна функция, которая определяет, что сообщение отправлено нажатием кнопки:

In [ ]:
@staticmethod
def is_button(content):
    return content.callback_query is not None and content.callback_query.data is not None

__Задание 3 (1 балл):__ Заменить кнопку возвращения изображения на две кнопки "Размытие" и "Оттенки серого", каждая из которых применяет соответствующее преобразование к сохраненному изображению и возвращает его пользователю. Загрузка нового изображения повторяет логику из предыдущего пункта, кнопки можно нажимать неограниченное количество раз

__Рекомендации:__

1. Указанные фильтры называются "gaussian smoothing" и "grayscale"

__Бонусное задание (до 5 баллов):__ Предложить и реализовать функциональные, эстетические или технические улучшения реализованного бота (допускается только в случае решения всех предыдущих пунктов задания)